<a href="https://colab.research.google.com/github/kavishsureka/C-A_capstone_project/blob/main/Capstone_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh pandas numpy


In [ ]:
from google.colab import files
uploaded = files.upload()  # upload your dataset.csv


Saving dataset.csv to dataset.csv


In [ ]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
output_notebook()


In [ ]:
df = pd.read_csv('dataset.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00


In [ ]:
def model_1_price(prev_price, occupancy, capacity, alpha=1.0):
    return prev_price + alpha * (occupancy / capacity)

df['price_model1'] = 10
for i in range(1, len(df)):
    prev_price = df.loc[i-1, 'price_model1']
    occupancy = df.loc[i, 'Occupancy']
    capacity = df.loc[i, 'Capacity']
    df.loc[i, 'price_model1'] = model_1_price(prev_price, occupancy, capacity)


/tmp/ipython-input-5-3487684541.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.1975' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'price_model1'] = model_1_price(prev_price, occupancy, capacity)


In [ ]:
p1 = figure(x_axis_type='datetime', title='Model 1: Price over Time')
p1.line(df['Timestamp'], df['price_model1'], color='blue', legend_label='Model 1')
p1.yaxis.axis_label = 'Price ($)'
p1.xaxis.axis_label = 'Time'
show(p1)


In [ ]:
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'cycle': 0.3, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

def compute_demand(row, alpha=1.0, beta=0.5, gamma=0.4, delta=0.6, epsilon=1.0):
    return (
        alpha * (row['Occupancy'] / row['Capacity']) +
        beta * row['QueueLength'] -
        gamma * (1 if row['TrafficConditionNearby'] == 'high' else 0) +
        delta * row['IsSpecialDay'] +
        epsilon * row['VehicleWeight']
    )

df['Demand'] = df.apply(compute_demand, axis=1)
df['DemandNorm'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())

df['price_model2'] = 10 * (1 + 0.8 * df['DemandNorm'])
df['price_model2'] = df['price_model2'].clip(lower=5, upper=20)


In [ ]:
p2 = figure(x_axis_type='datetime', title='Model 2: Price over Time')
p2.line(df['Timestamp'], df['price_model2'], color='green', legend_label='Model 2')
p2.yaxis.axis_label = 'Price ($)'
p2.xaxis.axis_label = 'Time'
show(p2)
